In [1]:
# Import the NetworkX package
import networkx as nx

import torch
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 1.10.2


In [2]:
!pwd

/Users/wangshihao/Research/SocialNetwork/HW1


In [3]:
# create BC dict
f = open('../data/hw1_data/Synthetic/5000/0_score.txt')
bc_dict = dict()
for line in f:
    line = line.replace("\n", "")
    a, b = line.split("\t")
    bc_dict[int(a)] = float(b)
f.close()
len(bc_dict.keys())

5000

In [4]:
f = open('../data/hw1_data/Synthetic/5000/0.txt')
text = []
for line in f:
    line = line.replace("\n", "")
    a, b = line.split("\t")
    text.append((int(a), int(b)))
f.close()

## 1. Graph

In [19]:
# Create an undirected graph G
G = nx.Graph(text)
print(G.is_directed())

False


In [20]:
# Get number of nodes
num_nodes = G.number_of_nodes()
print("G has {} nodes".format(num_nodes))

# Get number of edges
num_edges = G.number_of_edges()
print("G has {} edges".format(num_edges))

G has 5000 nodes
G has 19982 edges


In [21]:
node_id = 1

# Degree of node 1
print("Node {} has degree {}".format(node_id, G.degree[node_id]))

# Get neighbor of node 1
#for neighbor in G.neighbors(node_id):
#    print("Node {} has neighbor {}".format(node_id, neighbor))

Node 1 has degree 178


In [540]:
# set x feature
dergees = {node:[val, 1, 1] for (node, val) in G.degree()}
nx.set_node_attributes(G, dergees, "x")
nx.set_node_attributes(G, bc_dict, "y")
G.nodes[0]

{'x': [204, 1, 1], 'y': 0.0888865042991089}

In [608]:
class readData():
    def __init__(self, data):
        self.__bc_value = list()
        self.__text = list()
        self.__edges = list()
        
        
        if data == "youtube":
            pass
        else:
            path1 = f'../data/hw1_data/Synthetic/5000/{data}_score.txt'
            path2 = f'../data/hw1_data/Synthetic/5000/{data}.txt'
        
        # create BC dict
        f = open(path1)
        for line in f:
            line = line.replace("\n", "")
            n, bc = line.split("\t")
            self.__bc_value.append([int(n), math.log(float(bc)+1e-8)])
        f.close()
        
        # read node pair
        f = open(path2)
        a_list, b_list, Gtext = [], [], []
        for line in f:
            line = line.replace("\n", "")
            a, b = line.split("\t")
            a_list.append(int(a))
            b_list.append(int(b))
            Gtext.append((int(a), int(b)))
        self.__edges = [a_list+b_list, b_list+a_list]
        f.close()
        
        # Create an undirected graph G
        G = nx.Graph(Gtext)
        self.__nodes_list = [[val, 1, 1] for (node, val) in G.degree()]
        
        
    def get_nodes(self):
        return torch.Tensor(self.__nodes_list)

    def get_edges(self):
        return torch.tensor(self.__edges,dtype=torch.long)

    def get_bc_value(self):
        return torch.Tensor(self.__bc_value)
        
def read_Data(num):
    # create BC dict
    f = open(f'../data/hw1_data/Synthetic/5000/{num}_score.txt')
    bc_dict = dict()
    for line in f:
        line = line.replace("\n", "")
        a, b = line.split("\t")
        bc_dict[int(a)] = float(b)
    f.close()
    len(bc_dict.keys())
    
    # read node pair
    f = open(f'../data/hw1_data/Synthetic/5000/{num}.txt')
    text = []
    for line in f:
        line = line.replace("\n", "")
        a, b = line.split("\t")
        text.append((int(a), int(b)))
    f.close()
    
    # Create an undirected graph G
    G = nx.Graph(text)
    
    dergees = {node:[val, 1, 1] for (node, val) in G.degree()}
    nx.set_node_attributes(G, dergees, "x")
    nx.set_node_attributes(G, bc_dict, "y")
    
    return G

data = readData(0)
print(data.get_nodes().shape)
print(data.get_edges().shape)
print(data.get_bc_value())

torch.Size([5000, 3])
torch.Size([2, 39964])
tensor([[ 0.0000e+00, -2.3626e+00],
        [ 1.0000e+00, -2.9193e+00],
        [ 2.0000e+00, -3.1158e+00],
        ...,
        [ 4.9970e+03, -1.0842e+01],
        [ 4.9980e+03, -9.7114e+00],
        [ 4.9990e+03, -9.1234e+00]])


## 2. Pytorch Geometric

In [37]:
from torch_geometric.utils.convert import from_networkx
# Convert the graph into PyTorch geometric
pyg_graph = from_networkx(G)
print(pyg_graph)
print(pyg_graph.x)
print(pyg_graph.y)
print(pyg_graph.edge_index)
print(f'Number of features: {pyg_graph.num_features}')

data = pyg_graph  # Get the first graph object.

print(data)
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {(2*data.num_edges) / data.num_nodes:.2f}')

Data(x=[5000, 3], edge_index=[2, 39964], y=[5000])
tensor([[239,   1,   1],
        [196,   1,   1],
        [220,   1,   1],
        ...,
        [  4,   1,   1],
        [  4,   1,   1],
        [  4,   1,   1]])
tensor([9.4175e-02, 7.6438e-02, 9.2790e-02,  ..., 2.8613e-05, 2.5731e-05,
        2.0743e-05])
tensor([[   0,    0,    0,  ..., 4999, 4999, 4999],
        [   1,    2,    3,  ..., 2523, 4508, 4803]])
Number of features: 3
Data(x=[5000, 3], edge_index=[2, 39964], y=[5000])
Number of nodes: 5000
Number of edges: 39964
Average node degree: 15.99


## 2.1. Model

In [51]:
edge_index = data.edge_index
print(edge_index.t())

tensor([[   0,    1],
        [   0,    2],
        [   0,    3],
        ...,
        [4999, 2523],
        [4999, 4508],
        [4999, 4803]])


In [578]:
import torch
from torch.nn import GRU
from torch_geometric.nn import GCNConv

L = 5
    
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        torch.manual_seed(12345)
        self.input = GCNConv(pyg_graph.num_features, 128)
        self.relu = torch.nn.LeakyReLU()
        self.conv = GCNConv(128, 128)
        self.gru = GRU(128, 128, bias=False)
        self.linear = Linear(128, 64)
        self.output = Linear(64, 1)

    def forward(self, x, edge_index):
        # h1
        h1 = self.input(x, edge_index)
        h1 = self.relu(h1)
        hi_1 = torch.nn.functional.normalize(h1, p=2, dim=1)
        # h2 ~ L
        stack_h = hi_1.reshape(1, -1, 128)
        for epoch in range(L):
            #for i in range(x.shape[0]):
            # Caculate Neighbor
            # indices = torch.nonzero(edges[0]==i)
            # neighbor_index = edges[1,torch.flatten(indices)]
            # n_degree = pyg_graph.x[neighbor_index, 0]
            # v_degree = pyg_graph.x[i, 0] #x[i, 0]
            # dj = torch.sqrt(n_degree+1)
            # dv = torch.sqrt(v_degree+1)
            # normal_n = torch.div(hi_1[neighbor_index].reshape(128, -1), dv * dj)
            # h_n = torch.sum(normal_n, 1)
            h_n = self.conv(hi_1, edge_index)

            # GRU Cell
            hv = hi_1
            hv, _ = self.gru(h_n.reshape(1, *h_n.shape), hv.reshape(1, *hv.shape)) # (input, hidden)
            # concat GRU result
            # if i == 0:
            #     v_cat = hv
            # else: 
            #     v_cat = torch.cat([v_cat, hv], dim=0)

            # normalize new hi_1
            hi_1 = hv.reshape(-1, 128)
            hi_1 = torch.nn.functional.normalize(h1, p=2, dim=1)
            # store hidden result
            h_cat = torch.cat([stack_h, hi_1.reshape(1, -1, 128)], dim=0)
        # z is maximum from all hidden values
        z, _ = torch.max(h_cat, 0)
        
        # decorder
        h = self.linear(z)
        h = self.relu(h)
        out = self.output(h)
        
        return out

model = Model()
print(model)

Model(
  (input): GCNConv(3, 128)
  (relu): LeakyReLU(negative_slope=0.01)
  (conv): GCNConv(128, 128)
  (gru): GRU(128, 128, bias=False)
  (linear): Linear(in_features=128, out_features=64, bias=True)
  (output): Linear(in_features=64, out_features=1, bias=True)
)


In [544]:
pyg_graph.x.float()

tensor([[204.,   1.,   1.],
        [166.,   1.,   1.],
        [205.,   1.,   1.],
        ...,
        [  4.,   1.,   1.],
        [  4.,   1.,   1.],
        [  4.,   1.,   1.]])

In [532]:
y_hat= model(pyg_graph.x.float(), pyg_graph.edge_index)
print(y_hat)

tensor([[-0.1049],
        [-0.1050],
        [-0.1049],
        ...,
        [-0.1053],
        [-0.1056],
        [-0.1046]], grad_fn=<AddmmBackward0>)


In [522]:
from random import randint 

def sample_node(y_hat, edge_index, sample_num = 10000):
    loss = 0
    sigmoid = torch.nn.Sigmoid()
    for i in range(sample_num):
        idx = randint(0, edge_index.shape[1]-1)
        rand_edge = edge_index[:,idx]
        # loss
        yij = sigmoid(y_hat[rand_edge[0]] - y_hat[rand_edge[1]])
        bij = sigmoid(pyg_graph.y[rand_edge[0]] - pyg_graph.y[rand_edge[1]].reshape(1))
        if i == 0:
            yij_cat = yij
            bij_cat = bij
        else:
            yij_cat = torch.cat((yij_cat, yij))
            bij_cat = torch.cat((bij_cat, bij))
        #loss += -sigmoid(bij)*torch.log(sigmoid(yij)) - (1-sigmoid(bij))*torch.log(1-sigmoid(yij))
    
    return yij_cat, bij_cat
yij, bij = sample_node(y_hat, edges)
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
loss = criterion(yij.reshape(-1, 1), bij.reshape(-1, 1))

tensor(-0., grad_fn=<NegBackward0>)

In [599]:
import numpy as np
import networkx as nx
import random 
import math

class Graph():
    def __init__(self,batch_size):
        self.graph_list=[]
        self.__nodes_list = []
        self.__edge_index = []
        self.__bc_value = []
        self.__sample_pairs = []
        
        for x in range(batch_size):
            g = nx.powerlaw_cluster_graph(n=random.randint(150,200) , m=4, p=0.05, seed=None)
            self.graph_list.append(g)   
        
        s_list,t_list,en = [],[],0
        for g in self.graph_list:
            sample_1, sample_2 = [], []
            # BC Value
            self.__bc_value += list(nx.betweenness_centrality(g).values())
            # set nodes
            for x in range(g.number_of_nodes()):
                self.__nodes_list.append([g.degree[x],1,1])
                # sample pair
                sample_1 += [x+en for _ in range(5)]
                sample_2 += [x+en for _ in range(5)]
            random.shuffle(sample_1)
            random.shuffle(sample_2)
            self.__sample_pairs.extend([[i, j] for i,j in zip(sample_1,sample_2)])

            # set edges
            for e in g.edges():
                s,t = e
                s_list.append(s+en)
                t_list.append(t+en)
            en += g.number_of_nodes()
        self.__edge_index=[s_list+t_list,t_list+s_list]
        
        # log(BC value)
        for i, x in enumerate(self.__bc_value):
            self.__bc_value[i] = math.log(x+1e-8)
            
        
        
    def get_nodes(self):
        return torch.Tensor(self.__nodes_list)

    def get_edges(self):
        return torch.tensor(self.__edge_index,dtype=torch.long)

    def get_bc_value(self):
        return torch.Tensor(self.__bc_value)
        
    
    def get_pair_index(self):
        return torch.tensor(self.__sample_pairs, dtype=torch.long)
        


g = Graph(16)
print(g.get_nodes().shape)
print(g.get_edges().shape)
print(g.get_bc_value())
print(g.get_pair_index())

torch.Size([2916, 3])
torch.Size([2, 22792])
tensor([-2.6529, -1.8591, -3.5427,  ..., -6.7344, -7.1018, -7.0028])
tensor([[  75,   20],
        [ 145,   37],
        [  91,   13],
        ...,
        [2843, 2905],
        [2833, 2776],
        [2834, 2795]])


In [594]:
# training
import torch.nn.functional as F
from datetime import datetime

lr = 0.0001
MAX_ITERATION = 10000
model = Model()
criterion = torch.nn.CrossEntropyLoss(reduction='sum')  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def train():
    time_list = []
    for epoch in range(MAX_ITERATION):
        start = datetime.now()
        optimizer.zero_grad() # clear existing gradients
        if epoch % 500 == 0:
            G = Graph(16)
            bc = G.get_bc_value()
        # G = get_training_data()
        # pyg_graph = from_networkx(G)
        # edges = pyg_graph.edge_index

        # compute BC ranking score
        out= model(G.get_nodes(), G.get_edges())
        # compute loss
        node_pair = G.get_pair_index()
        y_hat = out[node_pair[:, 0]] - out[node_pair[:, 1]]
        grad = torch.sigmoid((bc[node_pair[:, 0]] - bc[node_pair[:, 1]]))
        #yij, bij = sample_node(y_hat, edges)
        loss = F.binary_cross_entropy_with_logits(y_hat, grad.reshape(-1, 1), reduction="sum")
        if epoch % 500 == 0:
            print("[{}/{}] Loss:{:.4f}".format(epoch, MAX_ITERATION, loss.item()))
        loss.backward()
        optimizer.step()
        end = datetime.now() - start
        time_list.append(end)
        
    torch.save(model.state_dict(), "./models/weight1.pth")
    print("Minimum time:{}".format(min(time_list)))
    print("Maximum time:{}".format(max(time_list)))

    return model

model = train()

[0/10000] Loss:9630.7988
[500/10000] Loss:8297.8584
[1000/10000] Loss:7246.6353
[1500/10000] Loss:6950.6426
[2000/10000] Loss:6965.8389
[2500/10000] Loss:6847.7969
[3000/10000] Loss:7089.0557
[3500/10000] Loss:7134.6812
[4000/10000] Loss:6713.6309
[4500/10000] Loss:6966.9185
[5000/10000] Loss:6991.6953
[5500/10000] Loss:7065.4434
[6000/10000] Loss:7091.1719
[6500/10000] Loss:7124.1362
[7000/10000] Loss:6996.0479
[7500/10000] Loss:7110.2715
[8000/10000] Loss:7077.5571
[8500/10000] Loss:7129.0552
[9000/10000] Loss:7050.8989
[9500/10000] Loss:6918.0205


In [635]:
# Top-N % accuracy
data = readData(0)
y_hat = model(data.get_nodes(), data.get_edges())
bc_value = data.get_bc_value()
predict_values =[[i, *j] for i, j in enumerate(y_hat.tolist())]

def topN_accuracy(predict_values ,bc_value, N):
    bc_value = sorted(bc_value, key = lambda e:e[1], reverse=True)
    predict_values = sorted(predict_values, key = lambda e:e[1], reverse=True)
    top_bc, top_predict = [], []
    for x in range(len(predict_values)*N//100):
        top_bc.append(int(bc_value[x][0]))
        top_predict.append(predict_values[x][0])
    
    return(len(set(top_bc)&set(top_predict)) / len(top_bc))

rank = [1, 5, 10]
for i in rank:
    print("Top {} accuracy: {}".format(i, topN_accuracy(predict_values, bc_value, N=i)))

Top 1 accuracy: 0.96
Top 5 accuracy: 0.82
Top 10 accuracy: 0.804


In [647]:
import scipy.stats as stats
# Kendall tau
data = readData(0)
y_hat = model(data.get_nodes(), data.get_edges())
bc_value = [ float(j) for i, j in data.get_bc_value()]
predict_values =[j[0] for i, j in enumerate(y_hat.tolist())]

# compute tau
tau, _ = stats.kendalltau(predict_values, bc_value)
print("Kendall tau: {}".format(tau))

Kendall tau: 0.5222393696530802
